<a href="https://colab.research.google.com/github/ChristineChung/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education',
    'NumCompaniesWorked', 'JobSatisfaction', 'OverTime',
    'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes

Age                         int64
DistanceFromHome            int64
Education                   int64
NumCompaniesWorked          int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train['OverTime'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

X_train['OverTime'].value_counts()

OverTime
0    780
1    322
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
from sklearn.preprocessing import OneHotEncoder
# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_encoded.shape[1], activation='sigmoid', name='department_output')(department_hidden_layer)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden_layer)

In [14]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 128)                  8320      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 32)                   4128      ['dense_1[0][0]']             
                                                                                              

In [17]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_encoded, 'attrition_output': y_train_attrition_encoded},
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded}),
    verbose=1
)

Epoch 1/100
35/35 [==============================] - 3s 25ms/step - loss: 1.3335 - department_output_loss: 0.8390 - attrition_output_loss: 0.4945 - department_output_accuracy: 0.6252 - attrition_output_accuracy: 0.8140 - val_loss: 1.1832 - val_department_output_loss: 0.8081 - val_attrition_output_loss: 0.3751 - val_department_output_accuracy: 0.6522 - val_attrition_output_accuracy: 0.8696
Epoch 2/100
35/35 [==============================] - 0s 12ms/step - loss: 1.1822 - department_output_loss: 0.7716 - attrition_output_loss: 0.4106 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8421 - val_loss: 1.1750 - val_department_output_loss: 0.8033 - val_attrition_output_loss: 0.3717 - val_department_output_accuracy: 0.6522 - val_attrition_output_accuracy: 0.8696
Epoch 3/100
35/35 [==============================] - 0s 14ms/step - loss: 1.1419 - department_output_loss: 0.7594 - attrition_output_loss: 0.3824 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.85

In [18]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)

# Print the evaluation list to understand its structure
print(evaluation)

# Extract the specific metrics
test_loss = evaluation[0]
department_output_loss = evaluation[1]
attrition_output_loss = evaluation[2]

print(f"Test Loss: {test_loss}")
print(f"Test Department Output Loss: {department_output_loss}")
print(f"Test Attrition Output Loss: {attrition_output_loss}")

12/12 [==============================] - 0s 4ms/step - loss: 3.8807 - department_output_loss: 2.4924 - attrition_output_loss: 1.3884 - department_output_accuracy: 0.5489 - attrition_output_accuracy: 0.8451
[3.8807272911071777, 2.4923593997955322, 1.3883678913116455, 0.5489130616188049, 0.845108687877655]
Test Loss: 3.8807272911071777
Test Department Output Loss: 2.4923593997955322
Test Attrition Output Loss: 1.3883678913116455


In [19]:
# Print the accuracy for both department and attrition

# Calculate accuracy for the department output
department_pred = model.predict(X_test_scaled)[0]
department_accuracy = np.mean(np.argmax(department_pred, axis=1) == np.argmax(y_test_encoded, axis=1))

# Calculate accuracy for the attrition output
attrition_pred = model.predict(X_test_scaled)[1]
attrition_accuracy = np.mean(np.argmax(attrition_pred, axis=1) == np.argmax(y_test_attrition_encoded, axis=1))

print(f"Test Department Output Accuracy: {department_accuracy}")
print(f"Test Attrition Output Accuracy: {attrition_accuracy}")

12/12 [==============================] - 0s 2ms/step
Test Department Output Accuracy: 0.5489130434782609
Test Attrition Output Accuracy: 0.845108695652174


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

### 1. Is accuracy the best metric to use on this data? Why or why not?

Accuracy can be a useful metric for evaluating the performance of a classification model, but it may not always be the best choice, especially if the dataset is imbalanced. In the context of employee attrition, if the dataset has a significantly higher number of employees who do not leave compared to those who do, accuracy might give a misleading sense of performance. In such cases, metrics like F1 score, precision, and recall are more informative as they consider the balance between false positives and false negatives. These metrics provide a better understanding of how well the model is performing in identifying the minority class (e.g., employees who leave).

### 2. What activation functions did you choose for your output layers, and why?

For the output layer, a sigmoid activation function was chosen. This is because the sigmoid function is suitable for binary classification tasks, as it outputs a probability value between 0 and 1, which can be interpreted as the likelihood of an employee leaving or staying. The choice of sigmoid makes sense for the department results because it allows for the possibility of multi-label classification, where an employee might belong to multiple departments or the company might add new departments in the future.

### 3. Can you name a few ways that this model might be improved?

Several strategies can be employed to improve the model:

1. **More Data**: Increasing the size of the training dataset can help the model learn better and generalize well to unseen data.

2. **Feature Engineering**: Creating new features or transforming existing ones to better capture the underlying patterns in the data can significantly improve model performance.

3. **Hyperparameter Tuning**: Adjusting the hyperparameters of the model, such as the learning rate, batch size, number of epochs, and architecture of the neural network (number of layers and neurons), can lead to better performance.

4. **Regularization**: Techniques like L1/L2 regularization or dropout can help prevent overfitting by penalizing large weights or randomly dropping neurons during training.

5. **Cross-Validation**: Using cross-validation techniques can provide a more robust estimate of model performance and help in selecting the best model.

6. **Ensemble Methods**: Combining multiple models through techniques like bagging, boosting, or stacking can lead to improved performance by leveraging the strengths of different models.

7. **Advanced Algorithms**: Exploring more advanced algorithms such as Gradient Boosting Machines (GBM), Random Forests, or even deep learning architectures like Convolutional Neural Networks (CNNs) or Recurrent Neural Networks (RNNs) if the data is suitable.

By implementing these strategies, the model's accuracy and overall performance can be enhanced, leading to more reliable predictions.
